In [1]:
import pymongo
import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize
from nltk import pos_tag
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict
from nltk.corpus import wordnet as wn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection, naive_bayes, svm
from sklearn.metrics import accuracy_score
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /home/brad/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [2]:
#set up server connection
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

In [3]:
np.random.seed(42)

In [4]:
db = client.Blurbs
uf = db.urban_fantasy

In [5]:
uf.count()

/home/brad/PythonData/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: count is deprecated. Use estimated_document_count or count_documents instead. Please note that $where must be replaced by $expr, $near must be replaced by $geoWithin with $center, and $nearSphere must be replaced by $geoWithin with $centerSphere
  """Entry point for launching an IPython kernel.


3751

In [6]:
uf.find_one()['blurb']

"When fifteen-year-old Clary Fray heads out to the Pandemonium Club in New York City, she hardly expects to witness a murder― much less a murder committed by three teenagers covered with strange tattoos and brandishing bizarre weapons. Then the body disappears into thin air. It's hard to call the police when the murderers are invisible to everyone else and when there is nothing―not even a smear of blood―to show that a boy has died. Or was he a boy?This is Clary's first meeting with the Shadowhunters, warriors dedicated to ridding the earth of demons. It's also her first encounter with Jace, a Shadowhunter who looks a little like an angel and acts a lot like a jerk. Within twenty-four hours Clary is pulled into Jace's world with a vengeance when her mother disappears and Clary herself is attacked by a demon. But why would demons be interested in ordinary mundanes like Clary and her mother? And how did Clary suddenly get the Sight? The Shadowhunters would like to know..."

In [7]:
results = uf.find()
blurbs = []
for result in results:
    blurbs.append({
        'genre' : 'Urban Fantasy',
        'blurb' : result['blurb']
    })

In [8]:
len(blurbs)

3751

In [9]:
ef = db.epic_fantasy
results = ef.find()
for result in results:
    blurbs.append({
        'genre' : 'Epic Fantasy',
        'blurb' : result['blurb']
    })

In [10]:
len(blurbs)

6627

In [11]:
sf = db.sci_fi
results = sf.find()
for result in results:
    blurbs.append({
        'genre' : 'Science Fiction',
        'blurb' : result['blurb']
    })

In [12]:
len(blurbs)

8947

In [13]:
thriller = db.thriller
results = thriller.find()
for result in results:
    blurbs.append({
        'genre' : 'Thriller',
        'blurb' : result['blurb']
    })

In [14]:
len(blurbs)

9405

In [15]:
rom = db.romance
results = rom.find()
for result in results:
    blurbs.append({
        'genre': 'Romance',
        'blurb' : result['blurb']
    })

In [ ]:
len(blurbs)

In [13]:
df = pd.DataFrame(blurbs)

In [14]:
df

,genre,blurb
0,Urban Fantasy,When fifteen-year-old Clary Fray heads out to ...
1,Urban Fantasy,"Mercedes Thompson, aka Mercy, is a talented Vo..."
2,Urban Fantasy,Harry Potter's life is miserable. His parents ...
3,Urban Fantasy,Only a true best friend can protect you from y...
4,Urban Fantasy,"Atlanta would be a nice place to live, if it w..."
...,...,...
8942,Science Fiction,When the government created 250 telepathic inf...
8943,Science Fiction,Discovered alone and without his memory on his...
8944,Science Fiction,New York Times Bestseller • Journey to Star Wa...
8945,Science Fiction,"Two people, who have been renamed Eiros and Ch..."


In [15]:
from sklearn.feature_extraction.text import CountVectorizer

In [16]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/brad/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [17]:
df['blurb'] = [blurb.lower() for blurb in df['blurb']]
blurbi = zip(df['blurb'].to_list())
print(blurbi)

In [18]:
vectorizer = CountVectorizer(max_features = 5000, min_df=3, max_df=0.7)


In [19]:
X = vectorizer.fit_transform(df['blurb'])
print(X)

  (0, 4853)	4
  (0, 1683)	1
  (0, 4976)	1
  (0, 3068)	1
  (0, 770)	6
  (0, 1793)	1
  (0, 2046)	1
  (0, 3122)	1
  (0, 795)	1
  (0, 2984)	1
  (0, 4983)	1
  (0, 755)	1
  (0, 3916)	1
  (0, 2015)	1
  (0, 1553)	1
  (0, 4913)	1
  (0, 2911)	2
  (0, 2905)	1
  (0, 2561)	1
  (0, 842)	1
  (0, 600)	2
  (0, 4412)	1
  (0, 4331)	1
  (0, 965)	1
  (0, 4910)	4
  :	:
  (8945, 1086)	1
  (8945, 3202)	2
  (8945, 4857)	2
  (8945, 3349)	1
  (8945, 420)	1
  (8945, 1434)	1
  (8945, 4560)	1
  (8945, 666)	1
  (8945, 4282)	1
  (8945, 750)	1
  (8945, 3152)	1
  (8945, 3426)	1
  (8945, 2643)	1
  (8945, 1237)	1
  (8945, 2113)	1
  (8945, 1555)	1
  (8945, 234)	1
  (8945, 4342)	1
  (8945, 246)	1
  (8945, 1719)	1
  (8945, 4024)	1
  (8946, 4394)	1
  (8946, 1749)	1
  (8946, 964)	1
  (8946, 3109)	1


In [20]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidfconverter = TfidfTransformer()
X = tfidfconverter.fit_transform(X).toarray()

In [21]:
y = df['genre']

In [22]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [23]:
from sklearn import model_selection, naive_bayes, svm
from sklearn.metrics import accuracy_score

In [36]:
SVM = svm.SVC(C=1.0, kernel='rbf', degree=3, gamma='auto', probability = True)
SVM.fit(X_train, y_train)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto', kernel='linear',
    max_iter=-1, probability=True, random_state=None, shrinking=True, tol=0.001,
    verbose=False)

In [37]:
predictions = SVM.predict(X_test)

In [38]:
accuracy_score(predictions, y_test)*100

75.3072625698324

In [39]:
blurb = [u"Zurcatte, a port city with moderately good trade importance, sees a trade windfall that sparks a struggle to maintain hold over the city. The tug between crown, council, gangs, and a vigilante group of murderers leaves the fate of the city and all involved up in the air."]

In [41]:
vectorizer = CountVectorizer()

In [44]:
X = vectorizer.fit_transform(blurb).toarray()

In [46]:
SVM.predict(X)

ValueError: X.shape[1] = 35 should be equal to 5000, the number of features at training time

In [ ]:
fftkov = "To escape from obscurity under the shadow of his heroic father, an arrogant thespian must unite a district of hardened gangsters and juvenile villains to start a revolution. But when a tech-savvy heroine hounds him at every corner, he realizes he must overcome her and his desire for his father’s approval by leading his army of outcasts as the King of Villains. Or he suffers the fate of all magical delinquents imprisoned at Power City: disappearing forever."